This is the template code of using GNN to solve DST problem instances from different types of graph with different scales.

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.13.1+cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 52.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import networkx as nx
import numpy as np
import random
import collections
import heapq
import copy
import math
import queue
from torch_geometric.data import Data
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
from torch.nn import Linear

In [ ]:
def generate_pages(num_pages):
    G = nx.erdos_renyi_graph(num_pages, 0.3, directed=True) # a random graph w.p. 0.3
    #G = nx.barabasi_albert_graph(num_pages, 10)
    #G= nx.scale_free_graph(num_pages, alpha=0.75, beta=0.2, gamma=0.05, create_using=X)
    adjacency = collections.defaultdict(set) # dictionary to store links of certain pages
    size = dict() # to store size of each page

    for i in G.nodes:
        for j in list(G.adj[i]):
            adjacency[i].add(j)
        size[i] = abs(random.gauss(10, 6)) # randomly assign weights to each node
    
    homepage = 0

    return homepage, adjacency, size

In [ ]:
def generate_pages(vertices, m): # random power law
    adjacency = collections.defaultdict(set) # dictionary to store links of certain pages
    outdegree = collections.defaultdict(int)
    size = dict() # to store size of each page
    a = 1.72

    for i in range(vertices): # id 0 to (vertices - 1)
        out_degree = math.ceil((np.random.pareto(a,) + 1)*m)
        while(out_degree > vertices):
            out_degree = math.ceil((np.random.pareto(a,) + 1)*m)  
        outdegree[i] = out_degree

        while(out_degree):
            link = random.randint(0, vertices - 1)
            if link != i and link not in adjacency[i]:
                adjacency[i].add(link)
                out_degree -= 1

    # max_node = max(outdegree, key=outdegree.get) 
    # adjacency[0], adjacency[max_node] =  adjacency[max_node], adjacency[0]
    # if 0 in adjacency[0]:
    #   adjacency[0].remove(0)
    #   adjacency[0].add(max_node)
    # if max_node in adjacency[max_node]:
    #   adjacency[max_node].remove(max_node)
    #   adjacency[max_node].add(0)

    for v in adjacency.keys():
        size[v] = round(abs(random.gauss(10, 6)), 2) # randomly assign size amount to each page
    
    homepage = 0

    return homepage, adjacency, size

In [ ]:
def Dijkstra(source):
    global size
    global adjacency
    
    linkedpages = set() # to store visited ones
    queue = [source]
    while(queue):
        page = queue.pop()
        linkedpages.add(page)
        for p in adjacency[page]:
            if p not in linkedpages:
                queue.append(p)
    
    dist = {}
    for page_id in linkedpages:
        dist[page_id] = [float('inf'), str()] # slower when adding path, maybe caused by list structure
    dist[source] = [0, str(source)]
    Q = [] # a priority queue
    S = set() # to store already visited page
    heapq.heappush(Q, (dist[source][0], source, dist[source][1]))
    N = len(linkedpages)
    
    while(len(S) < N):
        dis, page_id, path = heapq.heappop(Q)
        if page_id not in S:
            S.add(page_id)
            for next_page in adjacency[page_id]:
                if next_page not in S:
                    #dist[next_page][0] = min(dist[next_page][0], dis + size[next_page])
                    if dis + size[next_page] < dist[next_page][0]:
                        dist[next_page][0] = dis + size[next_page]
                        dist[next_page][1] = path + '-' + str(next_page)
                    heapq.heappush(Q, (dist[next_page][0], next_page, dist[next_page][1]))        
    
    return dist

In [ ]:
def subsets(terminals):
    allsubsets = []
    print(len(terminals))
    for i in range(int(math.pow(2, len(terminals)))):
        subset = []
        
        for j in range(len(terminals)):
            if (i&(1 << j) > 0):
                subset.append(terminals[j])
        
        allsubsets.append(subset)
    
    return allsubsets

In [ ]:
def Transition(S):
    visited = set()
    while(Q):
        dis, node = heapq.heappop(Q)
#         if node not in visited:
#             visited.add(node)
        for last_node in indegree[node]:
            if dp[node][S] + size[node] < dp[last_node][S]:
                tmp = set([node])
                for j in path[node][S]:
                     tmp.add(j)
                path[last_node][S] = tmp.copy()
                dp[last_node][S] = dp[node][S] +size[node]
                heapq.heappush(Q, (dp[last_node][S], last_node))
#             for next_node in adjacency[node]:
#                 if dp[next_node][S] + size[next_node] < dp[node][S]:
#                     dp[node][S] =  dp[next_node][S] + size[next_node]
#                     heapq.heappush(Q, (dp[node][S], node))

In [ ]:
class Net(torch.nn.Module): # GAT
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, h, last_h):
        super(Net, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        self.lins = torch.nn.ModuleList()

        self.convs.append(GATConv(in_channels, hidden_channels, heads=h))
        self.batch_norms.append(torch.nn.BatchNorm1d(h*hidden_channels))
        self.lins.append(Linear(in_channels, h*hidden_channels))        
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(h*hidden_channels, hidden_channels, heads=h))
            self.batch_norms.append(torch.nn.BatchNorm1d(h*hidden_channels))
            self.lins.append(Linear(h*hidden_channels, h*hidden_channels))        
        self.convs.append(GATConv(h*hidden_channels, out_channels, heads=last_h, concat=False))
        self.lins.append(Linear(h*hidden_channels, out_channels))

    def forward(self, x, edge_index):
        for conv, batch_norm, lin in zip(self.convs[:-1], self.batch_norms, self.lins[:-1]):
            x = conv(x, edge_index) + lin(x)
            x = batch_norm(x)
            x = F.relu(x)
            #x = F.dropout(x, p=0.2, training=self.training)
        
        x = self.convs[-1](x, edge_index) + self.lins[-1](x)
        x = torch.sigmoid(x)
        
        return x

In [ ]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = criterion(model(data.x, data.edge_index), data.y)
        total_loss += loss.item() * data.num_graphs        
        loss.backward()
        optimizer.step()

    return total_loss / len(train_loader.dataset)

In [ ]:
@torch.no_grad()
def test(loader):
    #global check_1, check_2
    model.eval()

    ys, preds = [], []
    count = 0
    for data in loader:
        out = model(data.x.to(device), data.edge_index.to(device))
        # check_1.append(data.y)
        # check_2.append(out)
        predicted = (out > 0.5).float().cpu()
        
        target_y = data.y.numpy().tolist()
        pred_y = predicted.numpy().tolist()

        ys.append(data.y)
        preds.append((out > 0.5).float().cpu())

        flag = 1
        for i in range(len(target_y)):
          if pred_y[i] != target_y[i]:
            flag = 0
        
        if flag:
          count += 1
    
    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()    
    node_count = 0
    for j in range(len(y)):
      if y[j] == pred[j]:
        node_count += 1

    return count / len(test_loader.dataset), node_count / len(y)

In [ ]:
scale = 100
homepage, adjacency, size = generate_pages(scale) # the scale is 100
#homepage, adjacency, size = generate_pages(scale, 2) # the scale is 100
k = 3
dist = Dijkstra(homepage)
indegree = collections.defaultdict(set)
for i in adjacency.keys():
    for j in adjacency[i]:
        indegree[j].add(i)

dest_set = []
result_set = []

num_samples = 40000

for _ in range(num_samples): # generate data 
    destinations = random.sample(dist.keys(), k) 
    while homepage in destinations or sorted(destinations) in dest_set:
        destinations = random.sample(dist.keys(), k)
    destinations = sorted(destinations)

    # initalize
    endstate = 1<<k
    dp = [[float('inf') for i in range(endstate)] for _ in range(scale)]
    count = 0
    Q = []
    for i in range(scale):
        if i in destinations:
            dp[i][1<<count] = 0
            count += 1

    path = [[set([i]) for _ in range(endstate)] for i in range(scale)]

    # DP
    for S in range(1, 1<<k):
        for i in range(scale):
            sub = (S-1)&S
            while(sub):
                if dp[i][sub]+dp[i][S^sub] < dp[i][S]:
                    dp[i][S] = dp[i][sub]+dp[i][S^sub]
                    tmp = set()
                    for j in path[i][sub]:
                        tmp.add(j)
                    for j in path[i][S^sub]:
                        tmp.add(j)
                    path[i][S] = tmp.copy()
                sub = (sub-1)&S

            if dp[i][S] < float('inf'):
                heapq.heappush(Q, (dp[i][S], i))

        Transition(S)

    result = dp[homepage][(1<<k)-1]
    guidance = path[homepage][-1]
    #print(guidance)
    result_set.append(guidance)
    dest_set.append(destinations)

In [ ]:
node_feature = [] # node-weighted
for i in range(len(dest_set)):
  min_size = min(size.values())
  max_size = max(size.values())
  features = [[(x-min_size)/(max_size-min_size), 0] for x in size.values()]
  features = [[x, 0] for x in size.values()]
  features[0][1] = 1
  for j in dest_set[i]:
    features[j][1] = 1
  node_feature.append(features)

In [ ]:
edges = []
for i in adjacency.keys():
  for j in adjacency[i]:
    edges.append([i, j])
edge_index = torch.tensor(edges, dtype=torch.long) # Edge connectivity with shape [2, num_edges]

In [ ]:
labels = [] # target labels
for i in range(len(result_set)):
  plain = [[0] for _ in range(scale)]
  plain[0] = [1]
  for j in result_set[i]:
    plain[j] = [1]
  
  labels.append(plain)

#y = torch.tensor(labels[0], dtype=torch.int64) # targets

In [ ]:
train_dataset = []
for i in range(int(num_samples*0.7)):
  x = torch.tensor(node_feature[i], dtype=torch.float) # Node feature matrix with shape [num_nodes, num_node_features (2 here, size, flag)]
  y = torch.tensor(labels[i], dtype=torch.float) # targets
  data = Data(x=x, y=y, edge_index=edge_index.t().contiguous()) # one set
  train_dataset.append(data)

In [ ]:
test_dataset = []
for i in range(int(num_samples*0.7), num_samples):
  x = torch.tensor(node_feature[i], dtype=torch.float) # Node feature matrix with shape [num_nodes, num_node_features (2 here, size, flag)]
  y = torch.tensor(labels[i], dtype=torch.float) # targets
  data = Data(x=x, y=y, edge_index=edge_index.t().contiguous()) # one set
  test_dataset.append(data)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # Adjust batch size to change RAM usage
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(2, 64, 1, 6, 4, 6).to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01) # Adam is faster but not stable 

In [ ]:
for epoch in range(100):
    loss = train()
    train_accuracy, train_node = test(train_loader) 
    test_accuracy, test_node = test(test_loader)
    print('Epoch: {:02d}, Loss: {:.4f}, Node accuracy of train: {:.4f}, Node accuracy of test: {:.4f}, Path accuracy of Test: {:.4f}'.format(
        epoch, loss, train_node, test_node, test_accuracy)) # the test_accuracy (of the entire graphs) is the metric of interest